# Class Project II - Data Wrangling from Data and Knowledge Sources (Patient Record, UMLS)

## My strategy:
<ul>
<li>Extract patient MRN and associated CUIs from XML file by iterating through each child node and using specific string names to extract its value</li>
<li>Filter and identify relationships between the MRCONSO and MRREL files and use pandas DataFrame to structure the data
<li>Merge identified relationships by CUI to extract desired data from each of the dataframes using pandas' left merge function, similar to a SQL left outer join</li>
<li>Convert final dataframe into a CSV output</li>
</ul>


### What is asked:
Create a CSV file with patient MRN, medication CUI, medication name, medication class, medication mechanism of action, and medication diagnosis as the columns

### Input:
<ul>
<li>Patient data as an XML file containing MRN and CUI of medications</li>
<li>UMLS (Unified Medical Language System) RRF files: MRCONSO.RRF (Concepts, Concept Names, and their sources), MRREL.RRF (Relationships)</li>
</ul>

### Output:
A CSV file containing patient MRN, medication CUI, medication name, medication class, medication mechanism of action, and medication diagnosis.

### Steps:
1. Import pandas and ElementTree modules
2. Parse XML file containing patient data
3. Iterate through each node of XML file
4. Extract patient MRN and associated medication CUIs; Store MRN as a dictionary key and CUIs in a list as dictionary values
5. Convert dictionary into a pandas DataFrame; Rename columns to MRN and CUI
6. Extract unique CUIs from dataframe
7. Load and read the MRCONSCO and MRREL files; Save contents into a list
8. Filter the MRREL list by the desired drug concepts IDs using a for loop
9. Split both MRCONSO and MRREL lists by "|" and save the contents into a new list.
10. Convert the lists into a pandas dataframe.
11. Filter the MRCONSO dataframe by desired drug concept IDs and save into a new dataframe; Keep columns containing concept ID and medication name; Rename the columns to CUI and Medication Name; Merge dataframes containing patient MRN and Medication Name by matching CUI using the pandas' left merge function 
12. Review the MRREL dataframe and identify the relationships with our desired drug concepts.
13. Filter the MRREL dataframe on the fifth column with our desired CUIs and store it into a new dataframe; Drop duplicate rows and keep only the columns containing the desired CUIs and relating drug concepts; Iterate through the new dataframe on the first column and match the CUI to the MRCONSO dataframe to find the medication class; Extract matched row from the MRCONSO dataframe into its own dataframe; Use the left merge function to combine the two dataframes by relating drug concepts; finally merge this by CUI to the final merged dataframe to include medication class
14. Filter the MRREL dataframe on the eighth column with 'mechanism_of_action_of' string and save it into a new dataframe; Drop duplicate rows and keep only the columns containing the desired CUIs and relating drug concepts; Iterate through the new dataframe on the fifth column and match the CUI to the MRCONSO dataframe to find the medication's mechanism of action; Extract matched row from the MRCONSO dataframe into its own dataframe; Use the left merge function to combine the two dataframes by relating drug concepts; finally merge this by CUI to the final merged dataframe to include medication's mechanism of action.
15. Filter the MRREL dataframe on the eighth column with 'may_be_treated_by' string and save it into a new dataframe; Keep only the columns containing the desired CUIs and relating drug concepts; Iterate through the new dataframe on the fifth column and match the CUI to the MRCONSO dataframe to find the diagnosis; Extract matched row from the MRCONSO dataframe into its own dataframe and keep only; Use the left merge function to combine the two dataframes by relating drug concepts; finally merge this by CUI to the final merged dataframe to include diagnosis treated by medication.
16. Review final merged dataframe
17. Export the final merged dataframe into a CSV file.

### Import Pandas and XML modules

In [1]:
import pandas as pd
from xml.etree import ElementTree as ET

### Parse XML file and get root element

In [2]:
tree = ET.parse('/opt/class/umls/med/dataset.xml')
root = tree.getroot()

### Iterating through XML tags

First we start with an empty dictionary to structure and store our patient MRNs and their medication CUI. This is done by iterating through each child node by specifying specific string names and extracting its value. The MRN is stored as the dictionary key and its associated CUIs as the values in the form of a list.

In [3]:
pat = {}

for parent in root:
    for child in parent:
        if child.tag == 'Identifier':
            mrn = child.find('MRN').get('value')
            
        if child.tag == 'Medication':
            # We store the associated CUIs in an empty list
            med = []
            for item in child.findall('Item'):
                med.append(item.get('value'))
    
    # We take the mrn as the key and and take the med list as its value
    pat[mrn] = med

In [4]:
# Reviewing the final dictionary output
pat

{'2XCKTA': ['C0012265', 'C0004147', 'C0025598'],
 '3GGA9I': ['C0025598', 'C0016860'],
 'FY4E1G': ['C0014025', 'C0012265', 'C0016365'],
 'KI9UDM': ['C0016365', 'C0012265'],
 'KZXQA6': ['C0016365', 'C0004147', 'C0025598', 'C0016860'],
 'O5KMNJ': ['C0014025', 'C0012265', 'C0016365', 'C0004147'],
 'Q1KQYR': ['C0004147', 'C0025598'],
 'UOV7U1': ['C0025598', 'C0016860'],
 'VH8BTT': ['C0014025', 'C0004147'],
 'ZIJ8GJ': ['C0014025', 'C0016365', 'C0016860']}

### Dictionary to pandas DataFrame

We convert our previous dictionary into a dataframe. The data is extracted using list comprehension.

In [5]:
df = pd.DataFrame([(mrn, cui) for mrn, val in pat.items() for cui in val], columns=['MRN','CUI'])

In [6]:
df

,MRN,CUI
0,VH8BTT,C0014025
1,VH8BTT,C0004147
2,KI9UDM,C0016365
3,KI9UDM,C0012265
4,UOV7U1,C0025598
5,UOV7U1,C0016860
6,FY4E1G,C0014025
7,FY4E1G,C0012265
8,FY4E1G,C0016365
9,KZXQA6,C0016365


### Reading through the RRF files

Next we are going to work on the MRCONSO and MRREL files by reading and saving its contents into a list

In [7]:
with open('/opt/class/umls/med/MRCONSO.RRF') as f:
    mrconso = (f.readlines())
    f.close()
    
with open('/opt/class/umls/med/MRREL.RRF') as f:
    mrrel_ = (f.readlines())
    f.close()

### Get unique CUIs

We obtain unique CUIs from our dataframe. This will allow us to iterate and filter through the MRCONSO and MRREL dataframes that we will later create.

In [8]:
cui_uniq = df['CUI'].unique()

### Filtering the MRREL list

We filter this list with our desired drug concepts from our unique CUIs that we extracted previously. This is accomplished by iterating through unique CUIs and MRREL list and extracting from the MRREL list if there is a match, which is then stored in a new list.

In [9]:
mrrel = []

for cui in cui_uniq:
    for s in mrrel_:
        if cui in s:
            mrrel.append(s)

### Iterating and splitting

We iterate through each line of the list and split the contents by "|" which are then saved into a new list.

In [10]:
mrconso_split = []
mrrel_split = []

for line in mrconso:
    mrconso_split.append(line.split('|'))

for line in mrrel:
    mrrel_split.append(line.split('|'))

### Converting to DataFrame

Next we will convert each list to a pandas dataframe to structure the data.

In [11]:
mrconso_df = pd.DataFrame(mrconso_split)
mrrel_df = pd.DataFrame(mrrel_split)

### Reviewing MRREL DataFrame

In [12]:
mrrel_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,C0014025,A17901948,AUI,RO,C0003873,A17992958,AUI,may_be_treated_by,R116110410,,NDFRT,NDFRT,,,N,,,\n
1,C0014025,A17901948,AUI,RO,C0011881,A18040373,AUI,may_be_treated_by,R129375635,,NDFRT,NDFRT,,,N,,,\n
2,C0014025,A17901948,AUI,RO,C2757044,A17995439,AUI,mechanism_of_action_of,R116200766,,NDFRT,NDFRT,,,N,,,\n
3,C0014025,A17901949,SCUI,PAR,C0012512,A17926076,SCUI,,R119383592,,NDFRT,NDFRT,,,N,,,\n
4,C0014025,A2881116,SCUI,PAR,C0003015,A3609518,SCUI,inverse_isa,R20277586,170783025,SNOMEDCT_US,SNOMEDCT_US,0,N,N,,,\n
5,C0003015,A3609517,SCUI,CHD,C0014025,A2881117,SCUI,isa,R20189582,1710977021,SNOMEDCT_US,SNOMEDCT_US,,Y,N,,,\n
6,C0004147,A18045083,AUI,RO,C0002962,A17966749,AUI,may_be_treated_by,R116142764,,NDFRT,NDFRT,,,N,,,\n
7,C0004147,A2878799,SCUI,PAR,C0304516,A3892946,SCUI,inverse_isa,R20275833,2463977027,SNOMEDCT_US,SNOMEDCT_US,0,N,N,,,\n
8,C0004147,A17949132,SCUI,PAR,C1563753,A17985216,SCUI,,R119383941,,NDFRT,NDFRT,,,N,,,\n
9,C0004147,A18045083,AUI,RO,C0004238,A17992698,AUI,may_be_treated_by,R116142766,,NDFRT,NDFRT,,,N,,,\n


### Filtering the MRCONSO dataframe

We will filter the dataframe by the specific drug concepts we are querying. We will only keep the columns that contain the CUI and medication name. We store this into a new dataframe and rename the columns.

In [13]:
med_name = mrconso_df[mrconso_df[0].isin(cui_uniq)][[0,14]]
med_name.columns = ['CUI','Medication_Name']

In [14]:
med_name

,CUI,Medication_Name
5,C0004147,Atenolol
16,C0012265,Digoxin
18,C0014025,Enalapril
19,C0016365,Fluoxetine
20,C0016860,Furosemide
27,C0025598,Metformin


### Merging medication name to patient MRN by CUI

We merge this dataframe to our first dataframe. The merge function is used on "left" by matching CUIs.

In [15]:
merged_df = df.merge(med_name, how = 'left', on = 'CUI')

In [16]:
merged_df

,MRN,CUI,Medication_Name
0,VH8BTT,C0014025,Enalapril
1,VH8BTT,C0004147,Atenolol
2,KI9UDM,C0016365,Fluoxetine
3,KI9UDM,C0012265,Digoxin
4,UOV7U1,C0025598,Metformin
5,UOV7U1,C0016860,Furosemide
6,FY4E1G,C0014025,Enalapril
7,FY4E1G,C0012265,Digoxin
8,FY4E1G,C0016365,Fluoxetine
9,KZXQA6,C0016365,Fluoxetine


### Get drug class

First we filter the MRREL dataframe on the fifth column with our desired CUIs and store it into a new dataframe. We drop duplicate rows and keep only the columns with our desired CUIs and relating drug concepts. The columns are renamed so we can later merge this dataframe by CUI.

In [17]:
get_drugclass = mrrel_df[mrrel_df[4].isin(cui_uniq)]
get_drugclass = get_drugclass.drop_duplicates(subset = 0)
get_drugclass = get_drugclass[[0,4]]
get_drugclass.columns = ['Rel', 'CUI']

In [18]:
get_drugclass

,Rel,CUI
5,C0003015,C0014025
11,C0304516,C0004147
16,C0360105,C0016365
25,C3536854,C0012265
31,C0005382,C0025598
33,C0354100,C0016860


### Match and merge

We iterate through relating drug concepts and match it the MRCONSO dataframe. We extract these rows from the MRCONSO dataframe and store it into a new dataframe. We keep only the columns that contain our relating drug concepts and medication class, then we rename the columns accordingly. Next we merge by relationship ('Rel') with our first dataframe.

In [19]:
# We create an empty dataframe to hold the matched observations by CUI
drugclass = pd.DataFrame()

for cui in get_drugclass['Rel']:
    drugclass = drugclass.append(mrconso_df[mrconso_df[0] == cui])

In [20]:
drugclass = drugclass[[0,14]]
drugclass.columns = ['Rel', 'Medication_Class']
drugclass

,Rel,Medication_Class
2,C0003015,ACE INHIBITORS
38,C0304516,Beta 1 adrenergic blocking agent
41,C0360105,Selective serotonin re-uptake inhibitor
54,C3536854,Cardiac Glycoside
6,C0005382,Biguanides
40,C0354100,LOOP DIURETICS


In [21]:
merged_drugclass = drugclass.merge(get_drugclass, how = 'left', on = 'Rel')
merged_drugclass

,Rel,Medication_Class,CUI
0,C0003015,ACE INHIBITORS,C0014025
1,C0304516,Beta 1 adrenergic blocking agent,C0004147
2,C0360105,Selective serotonin re-uptake inhibitor,C0016365
3,C3536854,Cardiac Glycoside,C0012265
4,C0005382,Biguanides,C0025598
5,C0354100,LOOP DIURETICS,C0016860


### Merging medication class to patient MRN by CUI

We merge our drug class dataframe to our final dataframe. The merge function is used on "left" by matching CUIs.

In [22]:
merged_df = merged_df.merge(merged_drugclass, how = 'left', on = 'CUI')
merged_df.drop('Rel', axis=1, inplace=True)
merged_df

,MRN,CUI,Medication_Name,Medication_Class
0,VH8BTT,C0014025,Enalapril,ACE INHIBITORS
1,VH8BTT,C0004147,Atenolol,Beta 1 adrenergic blocking agent
2,KI9UDM,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor
3,KI9UDM,C0012265,Digoxin,Cardiac Glycoside
4,UOV7U1,C0025598,Metformin,Biguanides
5,UOV7U1,C0016860,Furosemide,LOOP DIURETICS
6,FY4E1G,C0014025,Enalapril,ACE INHIBITORS
7,FY4E1G,C0012265,Digoxin,Cardiac Glycoside
8,FY4E1G,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor
9,KZXQA6,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor


### Get Mechanism of Action

First we filter the MRREL dataframe on the eighth column with string "mechanism_of_action_of" and store it into a new dataframe. We drop duplicate rows and keep only the columns with our desired CUIs and relating drug concepts. The columns are renamed so we can later merge this dataframe by CUI.

In [23]:
get_mech = mrrel_df[(mrrel_df[7] == 'mechanism_of_action_of')]
get_mech = get_mech[[0, 4]]
get_mech = get_mech.drop_duplicates(subset = 0)
get_mech.columns = ['CUI', 'Rel']

In [24]:
get_mech

,CUI,Rel
2,C0014025,C2757044
10,C0004147,C1373134
14,C0016365,C2757053
19,C0012265,C1373085
30,C0025598,C1373092
37,C0016860,C2936809


### Match and merge

We iterate through relating drug concepts and match it the MRCONSO dataframe. We extract these rows from the MRCONSO dataframe and store it into a new dataframe. We keep only the columns that contain our relating drug concepts and mechanism of action, then we rename the columns accordingly. Next we merge by relationship ('Rel') with our first dataframe.

In [25]:
# We create an empty dataframe to hold the matched observations by CUI
mech = pd.DataFrame()

for cui in get_mech['Rel']:
    mech = mech.append(mrconso_df[mrconso_df[0] == cui])

In [26]:
mech = mech[[0, 14]]
mech.columns = ['Rel', 'Mechanism_of_Action']
mech

,Rel,Mechanism_of_Action
50,C2757044,Angiotensin Converting Enzyme Inhibitor
47,C1373134,Adrenergic beta-1 Antagonists
51,C2757053,Serotonin Uptake Inhibitors
45,C1373085,Sodium-Potassium Exchanging ATPase Interactions
46,C1373092,Insulin Receptor Agonists
52,C2936809,Sodium Potassium Chloride Symporter Inhibitors


In [27]:
merged_mech = mech.merge(get_mech, how = 'left', on = 'Rel')
merged_mech

,Rel,Mechanism_of_Action,CUI
0,C2757044,Angiotensin Converting Enzyme Inhibitor,C0014025
1,C1373134,Adrenergic beta-1 Antagonists,C0004147
2,C2757053,Serotonin Uptake Inhibitors,C0016365
3,C1373085,Sodium-Potassium Exchanging ATPase Interactions,C0012265
4,C1373092,Insulin Receptor Agonists,C0025598
5,C2936809,Sodium Potassium Chloride Symporter Inhibitors,C0016860


### Merging mechanism of action to patient MRN by CUI

We merge our mechanism of action dataframe to our final dataframe. The merge function is used on "left" by matching CUIs.

In [28]:
merged_df = merged_df.merge(merged_mech, how = 'left', on = 'CUI')
merged_df.drop('Rel', axis=1, inplace=True)
merged_df

,MRN,CUI,Medication_Name,Medication_Class,Mechanism_of_Action
0,VH8BTT,C0014025,Enalapril,ACE INHIBITORS,Angiotensin Converting Enzyme Inhibitor
1,VH8BTT,C0004147,Atenolol,Beta 1 adrenergic blocking agent,Adrenergic beta-1 Antagonists
2,KI9UDM,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor,Serotonin Uptake Inhibitors
3,KI9UDM,C0012265,Digoxin,Cardiac Glycoside,Sodium-Potassium Exchanging ATPase Interactions
4,UOV7U1,C0025598,Metformin,Biguanides,Insulin Receptor Agonists
5,UOV7U1,C0016860,Furosemide,LOOP DIURETICS,Sodium Potassium Chloride Symporter Inhibitors
6,FY4E1G,C0014025,Enalapril,ACE INHIBITORS,Angiotensin Converting Enzyme Inhibitor
7,FY4E1G,C0012265,Digoxin,Cardiac Glycoside,Sodium-Potassium Exchanging ATPase Interactions
8,FY4E1G,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor,Serotonin Uptake Inhibitors
9,KZXQA6,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor,Serotonin Uptake Inhibitors


### Get diagnosis

First we filter the MRREL dataframe on the eighth column with string "may_be_treated_by" and store it into a new dataframe. We drop duplicate rows and keep only the columns with our desired CUIs and relating drug concepts. The columns are renamed so we can later merge this dataframe by CUI.

In [29]:
get_dx = mrrel_df[(mrrel_df[7] == 'may_be_treated_by')]
get_dx = get_dx[[0,4]]
get_dx.columns = ['CUI', 'Rel']

In [30]:
get_dx

,CUI,Rel
0,C0014025,C0003873
1,C0014025,C0011881
6,C0004147,C0002962
9,C0004147,C0004238
12,C0016365,C0006370
13,C0016365,C0011581
17,C0012265,C0004238
18,C0012265,C0039240
26,C0025598,C0011860
35,C0016860,C0020538


### Match and merge

We iterate through relating drug concepts and match it the MRCONSO dataframe. We extract these rows from the MRCONSO dataframe and store it into a new dataframe. We keep only the columns that contain our relating drug concepts and diagnosis, then we rename the columns accordingly. Next we merge by relationship ('Rel') with our first dataframe.

In [31]:
# We create an empty dataframe to hold the matched observations by CUI
dx = pd.DataFrame()

for cui in get_dx['Rel']:
    dx = dx.append(mrconso_df[mrconso_df[0] == cui])

In [32]:
dx = dx[[0,14]]
dx.columns = ['Rel', 'Medication_Diagnosis']
dx

,Rel,Medication_Diagnosis
14,C0011881,Diabetic nephropathy
1,C0002962,Angina pectoris
11,C0011581,Depressive disorder
34,C0039240,Supraventricular tachycardia
13,C0011860,Type 2 diabetes mellitus
32,C0034063,Pulmonary edema


In [33]:
merged_dx = dx.merge(get_dx, how = 'left', on = 'Rel')
merged_dx

,Rel,Medication_Diagnosis,CUI
0,C0011881,Diabetic nephropathy,C0014025
1,C0002962,Angina pectoris,C0004147
2,C0011581,Depressive disorder,C0016365
3,C0039240,Supraventricular tachycardia,C0012265
4,C0011860,Type 2 diabetes mellitus,C0025598
5,C0034063,Pulmonary edema,C0016860


### Merging diagnosis to patient MRN by CUI

We merge our diagnosis dataframe to our final dataframe. The merge function is used on "left" by matching CUIs.
Finally we review our final dataframe.

In [34]:
merged_df = merged_df.merge(merged_dx, how = 'left', on = 'CUI')
merged_df.drop('Rel', axis=1, inplace=True)
merged_df

,MRN,CUI,Medication_Name,Medication_Class,Mechanism_of_Action,Medication_Diagnosis
0,VH8BTT,C0014025,Enalapril,ACE INHIBITORS,Angiotensin Converting Enzyme Inhibitor,Diabetic nephropathy
1,VH8BTT,C0004147,Atenolol,Beta 1 adrenergic blocking agent,Adrenergic beta-1 Antagonists,Angina pectoris
2,KI9UDM,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor,Serotonin Uptake Inhibitors,Depressive disorder
3,KI9UDM,C0012265,Digoxin,Cardiac Glycoside,Sodium-Potassium Exchanging ATPase Interactions,Supraventricular tachycardia
4,UOV7U1,C0025598,Metformin,Biguanides,Insulin Receptor Agonists,Type 2 diabetes mellitus
5,UOV7U1,C0016860,Furosemide,LOOP DIURETICS,Sodium Potassium Chloride Symporter Inhibitors,Pulmonary edema
6,FY4E1G,C0014025,Enalapril,ACE INHIBITORS,Angiotensin Converting Enzyme Inhibitor,Diabetic nephropathy
7,FY4E1G,C0012265,Digoxin,Cardiac Glycoside,Sodium-Potassium Exchanging ATPase Interactions,Supraventricular tachycardia
8,FY4E1G,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor,Serotonin Uptake Inhibitors,Depressive disorder
9,KZXQA6,C0016365,Fluoxetine,Selective serotonin re-uptake inhibitor,Serotonin Uptake Inhibitors,Depressive disorder


### Writing as CSV output

In [35]:
merged_df.to_csv('jbautista2_cp2.csv', index=False)